# TF fuction 和 AutoGraph

要获得最佳性能并使模型可在任何地方部署，请使用tf.function从程序中构建图。 因为有AutoGraph，可以使用tf.function构建高效性能的Python代码，但仍有一些陷阱需要警惕。

下面的辅助程序代码，用于演示可能遇到的各种错误。

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
print('tf version:', tf.__version__)

tf version: 2.0.0-rc0


In [1]:
import contextlib

# 构建包含上下文管理器的函数，使其可以在with中使用
@contextlib.contextmanager
def assert_raises(error_class):
    try:
        yield
    except error_class as e:
        print('Caught expected exception \n  {}: {}'.format(error_class, e))
    except Exception as e:
        print('Got unexpected exception \n  {}: {}'.format(type(e), e))
    else:
        raise Exception('Expected {} to be raised but no error was raised!'.format(
            error_class))

一个tf.function定义就像是一个核心TensorFlow操作：可以急切地执行它; 也可以在图表中使用它; 它有梯度; 等等。

In [4]:
# 类似一个tensorflow操作
@tf.function
def add(a, b):
    return a + b

add(tf.ones([2,2]), tf.ones([2,2]))

<tf.Tensor: id=14, shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [5]:
# tf.function操作可以计算梯度
@tf.function
def add(a, b):
    return a + b
v = tf.Variable(2.0)
with tf.GradientTape() as tape:
    res = add(v, 1.0)

tape.gradient(res, v)

<tf.Tensor: id=38, shape=(), dtype=float32, numpy=1.0>

In [6]:
# 可以内嵌调用tf.function
@tf.function
def dense_layer(x, w, b):
    return add(tf.matmul(x, w), b)

dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: id=64, shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

## 跟踪和多态

Python 的动态类型意味着可以使用各种参数类型调用函数，Python 将在每个场景中执行不同的操作。

另一方面，TensorFlow 图需要静态 dtypes 和形状尺寸。tf.function 通过在必要时回溯函数来生成正确的图形来弥补这一差距。大多数使用的微妙 tf.function 源于这种回归行为。

您可以使用不同类型的参数调用函数来查看正在发生的事情。

In [7]:
# 函数的多态
@tf.function
def double(a):
    print('追踪变量：',a)
    return a + a

print('结果:',double(tf.constant(1)))
print()
print('结果:',double(tf.constant(1.1)))
print()
print('结果:',double(tf.constant('c')))

追踪变量： Tensor("a:0", shape=(), dtype=int32)
结果: tf.Tensor(2, shape=(), dtype=int32)

追踪变量： Tensor("a:0", shape=(), dtype=float32)
结果: tf.Tensor(2.2, shape=(), dtype=float32)

追踪变量： Tensor("a:0", shape=(), dtype=string)
结果: tf.Tensor(b'cc', shape=(), dtype=string)


控制参数类型： 创建一个新的 tf.function。tf.function 确保单独的对象不共享跟踪。 使用该 get_concrete_function 方法获取特定追踪 指定 input_signature 何时调用 tf.function 以确保仅构建一个功能图。

In [8]:
print('构建许可的追踪')
double_strings = double.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string))
print("执行追踪函数")
print(double_strings(tf.constant("a")))
print(double_strings(a=tf.constant("b")))
print("使用不合法参数")
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1))

构建许可的追踪
追踪变量： Tensor("a:0", dtype=string)
执行追踪函数
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)
使用不合法参数
Caught expected exception 
  <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>: cannot compute __inference_double_91 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_91]


## 什么时候回溯？

多态tf.function通过跟踪生成具体函数的缓存。缓存键实际上是从函数args和kwargs生成的键的元组。为tf.Tensor参数生成的关键是其形状和类型。为Python原语生成的密钥是它的值。对于所有其他Python类型，键都基于对象，id()以便为每个类的实例独立跟踪方法。将来，TensorFlow可以为Python对象添加更复杂的缓存，可以安全地转换为张量。

## 使用Python参数还是Tensor参数s？

通常，Python的参数被用来控制超参数和图形的结构-例如，num_layers=10或training=True或nonlinearity='relu'。因此，如果Python参数发生变化，那么必须回溯图。

但是，Python参数可能不会用于控制图构造。在这些情况下，Python值的变化可能会触发不必要的回溯。举例来说，这个训练循环，AutoGraph将动态展开。尽管存在多条迹线，但生成的图实际上是相同的，因此这有点低效。

In [9]:
def train_one_step():
    pass

@tf.function
def train(num_steps):
    print("追踪： num_steps = {}".format(num_steps))
    for _ in tf.range(num_steps):
        train_one_step()

train(num_steps=10)
train(num_steps=20)

追踪： num_steps = 10
追踪： num_steps = 20


In [10]:
# 使用tensor，同类型不会重复追踪
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))

追踪： num_steps = Tensor("num_steps:0", shape=(), dtype=int32)


In [11]:
# 使用tensor，类型不同才会有新的追踪，（前一个单元格已追踪int型，所以该处不追踪）
train(num_steps=tf.constant(10, dtype=tf.int32))
train(num_steps=tf.constant(20.6))

追踪： num_steps = Tensor("num_steps:0", shape=(), dtype=float32)


## 副作用 tf.function

通常，Python副作用（如打印或变异对象）仅在跟踪期间发生。怎么能可靠地触发副作用tf.function呢？

一般的经验法则是仅使用Python副作用来调试跟踪。但是，TensorFlow操作类似于tf.Variable.assign，tf.print并且tf.summary是确保TensorFlow运行时在每次调用时跟踪和执行代码的最佳方法。通常使用功能样式将产生最佳结果。

tf.function函数中的print()被用于跟踪，所以要调试输出每次调用(副作用),就需要tf.function()

In [12]:
@tf.function
def f(x):
    print("追踪：", x)
    tf.print('执行：', x)
f(1)
f(1)
f(2)

追踪： 1
执行： 1
执行： 1
追踪： 2
执行： 2


如果想在每次调用期间执行Python代码tf.function，可以使用tf.py_function。tf.py_function缺点是它不便携和高效，也不能在分布式（多GPU，TPU）设置中很好地工作。此外，由于tf.py_function必须连接到图，它将所有输入/输出转换为张量。

In [13]:
external_list = []

def side_effect(x):
    print('Python side effect')
    external_list.append(x)

@tf.function
def f(x):
    tf.py_function(side_effect, inp=[x], Tout=[])

f(1)
f(1)
f(1)
print(external_list)

Python side effect
Python side effect
Python side effect
[<tf.Tensor: id=275, shape=(), dtype=int32, numpy=1>, <tf.Tensor: id=276, shape=(), dtype=int32, numpy=1>, <tf.Tensor: id=277, shape=(), dtype=int32, numpy=1>]


详细内容参见 [参考](https://zhuanlan.zhihu.com/p/72622208)